In [14]:
from __future__ import annotations

import numpy as np
import jax.numpy as jnp
import pandas as pd
import polars as pl
import seaborn as sns
from scipy.stats import binom
from nptyping import NDArray, Shape, Int, Float, Bool
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import display, HTML
from rich import print


def load_mathjax():
    """Load MathJax library in JupyterLab to enable LaTeX rendering in Plotly charts.

    This function checks if the MathJax library is already loaded, and if not,
    it loads the library from the provided CDN link.
    """
    display(HTML("""
        <script>
            if (typeof MathJax === 'undefined') {
                var script = document.createElement('script');
                script.type = 'text/javascript';
                script.src = 'https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.7/MathJax.js?config=TeX-AMS-MML_HTMLorMML';
                document.head.appendChild(script);
            }
        </script>
    """))
# Load MathJax
load_mathjax()

In [31]:
def generate_covariance_matrix(condition_number, initial_axis):
    """Generate a random covariance matrix and its inverse with the given condition number
    and the given first axis (unnormalized). The matrix is generated using Householder transformation.

    Parameters
    ----------
    condition_number : float
        The desired condition number for the generated covariance matrix.
    initial_axis : numpy.ndarray, shape (D, 1)
        The initial axis vector used for generating the covariance matrix.
        The shape of the array should be (D, 1), where D is the number of dimensions.

    Returns
    -------
    covariance_matrix : numpy.ndarray, shape (D, D)
        The generated random covariance matrix with the given condition number.
    inverse_covariance_matrix : numpy.ndarray, shape (D, D)
        The inverse of the generated covariance matrix.
    """
    initial_axis = np.copy(initial_axis)
    initial_axis[0] += np.sign(initial_axis[1]) * np.linalg.norm(initial_axis)
    # Householder transformation
    basis = np.eye(len(initial_axis)) - 2 * np.outer(initial_axis, initial_axis) / np.linalg.norm(initial_axis) ** 2

    eigenvalues = np.flip(np.sort(1.0 / np.linspace(condition_number, 1, len(initial_axis))))
    covariance_matrix = basis @ np.diag(eigenvalues) @ basis.T
    inverse_covariance_matrix = basis @ np.diag(1 / eigenvalues) @ basis.T

    return covariance_matrix, inverse_covariance_matrix

def compute_evals(dimensions, covariance_matrix, num_samples):
    """Compute the eigenvalues and covariance matrices of the MLE and MAP estimates
    for a given covariance matrix and number of samples.

    Parameters
    ----------
    dimensions : int
        The number of dimensions for the covariance matrix.
    covariance_matrix : numpy.ndarray, shape (dimensions, dimensions)
        The true covariance matrix to generate samples from.
    num_samples : int
        The number of samples to generate for estimating eigenvalues and covariance matrices.

    Returns
    -------
    mle_evals : numpy.ndarray, shape (dimensions,)
        The eigenvalues of the maximum likelihood estimate (MLE) covariance matrix.
    map_evals : numpy.ndarray, shape (dimensions,)
        The eigenvalues of the maximum a posteriori (MAP) estimate covariance matrix.
    mle_covariance : numpy.ndarray, shape (dimensions, dimensions)
        The maximum likelihood estimate (MLE) covariance matrix.
    map_covariance : numpy.ndarray, shape (dimensions, dimensions)
        The maximum a posteriori (MAP) estimate covariance matrix.
    """
    samples = np.random.multivariate_normal(np.zeros(dimensions), covariance_matrix, num_samples)

    mle_covariance = np.cov(samples, rowvar=False, bias=True)
    mle_evals = np.real(np.flip(np.sort(np.linalg.eig(mle_covariance)[0])))

    shrinkage_lambda = 0.9
    map_covariance = shrinkage_lambda * np.diag(np.diag(mle_covariance)) + (1 - shrinkage_lambda) * mle_covariance
    map_evals = np.real(np.flip(np.sort(np.linalg.eig(map_covariance)[0])))

    return mle_evals, map_evals, mle_covariance, map_covariance


def plot_eigenvalues_comparison(dimensions, covariance_matrix, fractions, seed=42):
    """Plot a comparison of eigenvalues for true, MLE, and MAP covariance matrices.

    Parameters
    ----------
    dimensions : int
        The number of dimensions for the covariance matrix.
    covariance_matrix : numpy.ndarray, shape (dimensions, dimensions)
        The true covariance matrix to generate samples from.
    fractions : list of float
        The fractions of the number of dimensions to determine the number of samples.
    seed : int, optional, default: 42
        The random seed for reproducibility.
    """
    np.random.seed(seed)
    true_evals = np.flip(np.sort(np.linalg.eig(covariance_matrix)[0]))

    for fraction in fractions:
        fig = go.Figure()

        num_samples = int(fraction * dimensions)
        mle_evals, map_evals, mle_covariance, map_covariance = compute_evals(dimensions, covariance_matrix, num_samples)
        
        eigenvalues = [true_evals, mle_evals, map_evals]
        covariances = [covariance_matrix, mle_covariance, map_covariance]
        colors = ['black', 'blue', 'red']
        line_types = ['solid', 'dot', 'dashdot']
        names = ['True', 'MLE', 'MAP']

        for evals, covariances, color, line_type, name in zip(eigenvalues, covariances, colors, line_types, names):
            trace = go.Scatter(
                x=np.arange(1, dimensions + 1),
                y=evals,
                mode="lines",
                line=dict(color=color, width=3, dash=line_type),
                name=f"{name}, k={np.linalg.cond(covariances):.2e}"
            )
            fig.add_trace(trace)

        fig.update_layout(title=f'N={num_samples}, D={dimensions}')
        fig.update_xaxes(title_text="Eigenvalue", range=[0, 30])
        fig.update_yaxes(title_text="Value", range=[0-0.1, 1.5+0.1])
        fig.show()


dimensions = 50
condition_number = 10
initial_axis = np.random.randn(dimensions, 1)
covariance_matrix, _ = generate_covariance_matrix(condition_number, initial_axis)
fractions = [2, 1, 0.5]

plot_eigenvalues_comparison(dimensions, covariance_matrix, fractions)